In [1]:
import sqlite3
import pandas as pd
import os
import sys
import math
import numpy as np

def fasta2List(pathFasta):
    f = open(pathFasta, "r")
    title = []
    seq = []
    seq_temp = []
    for line in f:
        if line[0] == ">":
            seq.append(''.join(seq_temp).replace("\n", ""))
            title.append(line.replace("\n", ""))
            seq_temp = []
        else:
            seq_temp.append(line)
    seq.append(''.join(seq_temp).replace("\n", ""))
    seq.pop(0)
    dictionary = dict(zip(title, seq))
    return dictionary


conn = sqlite3.connect('../../mismatch_db.db')
mismatch = pd.read_sql_query("SELECT * FROM mismatch", conn)
mismatch = mismatch.astype({"exon_start_prim": "Int64", "exon_stop_prim":"Int64", "exon_start_hum": "Int64", "exon_stop_hum":"Int64"})

prim_exon_introns = pd.read_sql_query("""
SELECT mismatch_ID, mismatch.prot_prim, exon_intron_map.'type', exon_intron_map.number_elem, exon_intron_map.seq
FROM mismatch
JOIN protein ON mismatch.prot_prim = protein.prot_ID
JOIN transcript ON protein.transcript_ID = transcript.transcript_ID
JOIN exon_intron_map ON transcript.transcript_ID = exon_intron_map.transcript_ID
""", conn)


In [34]:
out_folder = "../../data/mismatch-analysis/tblastn2/"
raw_folder = "../../data/raw/uniprot-blast/results/"
os.mkdir(out_folder)
os.mkdir(out_folder+"/query_subject")
os.mkdir(out_folder+"/blast_out")

In [37]:
fff = open(out_folder+"/all_couple.txt", "w")
for index, row in mismatch.iloc[:, :].iterrows():
    fasta_name = row[1]
    prot_name = row[2]
    peptide_Ref = row[12]
    try:
        prim_exon = prim_exon_introns.loc[(prim_exon_introns["mismatch_ID"]==row[0]) & (prim_exon_introns["number_elem"].isin(range(row[7],row[8]+1))) & (prim_exon_introns["type"]=="Exon")]
        prim_intron = prim_exon_introns.loc[(prim_exon_introns["mismatch_ID"]==row[0]) & (prim_exon_introns["number_elem"].isin(range(row[7],row[8]))) & (prim_exon_introns["type"]=="Intron")]
    except:
        continue
    genomic_Seq = ""
    for i in range(0,row[8]+1-row[7]):
        genomic_Seq += prim_exon.iloc[i,4]
        try: genomic_Seq += prim_intron.iloc[i,4]
        except: pass

    f = open(out_folder+"/query_subject/"+str(index)+"_" +
             str(row[1])+"_"+str(prot_name)+".subject", "w")
    ff = open(out_folder+"/query_subject/"+str(index)+"_" +
              str(row[1])+"_"+str(prot_name)+".query", "w")

    f.write(">"+prot_name+"\n"+str(genomic_Seq))
    ff.write(">"+row[1]+"\n"+str(peptide_Ref))
    fff.write(str(index)+"_" +
              str(row[1])+"_"+str(prot_name)+"\n")

    f.close()
    ff.close()
fff.close()